#SQLCoder-7b-2


##Setup

In [1]:
!pip install torch transformers bitsandbytes accelerate sqlparse




















     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 32.6 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
torch.cuda.is_available()

True

In [4]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [5]:
print(available_memory)

15835660288


##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [6]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [7]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE [PowerAppDev].[tblOPPERTUNITIESTest](

[OppertunityID] [int] NOT NULL,

[OppertunityIDForSearch] [varchar](20) NULL,

[OppertunityName] [varchar](150) NOT NULL,

[Created] [datetime2](6) NULL,

[ESTIMATE] [decimal](13, 2) NOT NULL,

[ESTCLOSE_DATE] [datetime2](6) NULL,

[YearQuarter] [varchar](7) NULL,

[Antcipated] [numeric](20, 5) NULL,

[SalesRep] [char](10) NOT NULL,

[SEGMENT_UID] [int] NOT NULL,

[Segment] [varchar](100) NULL,

[ACCOUNT_UID] [int] NOT NULL,

[AccountName] [varchar](100) NULL,

[SALESSTAGE_UID] [int] NOT NULL,

[Stage] [varchar](100) NULL,

[ProjectStage] [varchar](200) NULL,

[SQUAREFOOTAGE] [decimal](13, 2) NULL,

[ProductLevelId] [int] NULL,

[Probability] [numeric](6, 3) NULL,

[SamplesSent] [varchar](10) NULL,

[Specified] [varchar](10) NULL,

[InvoicePrice] [float] NULL,

[QuanitityPrice] [float] NULL,

[SubmittalsStatus] [varchar](200) NULL,

[OutOfRegion] [bit] NULL,

[Notes] [varchar](2000) NULL,

[LastUpdatedBy] [varchar](200) NULL,

[LastUpdatedDate] [datetime] NULL,

[LastUpdatedDateFromPowerApp] [datetime] NULL,

[Description] [varchar](150) NULL,

[PremierFlag] [bit] NULL,

[AddressID] [varchar](20) NULL,

[LostReasonTypeId] [int] NULL,

[ADDRESS1] [varchar](150) NULL,

[ADDRESS2] [varchar](150) NULL,

[CITY] [varchar](150) NULL,

[STATE] [char](2) NULL,

[COUNTRYCODE] [char](2) NULL,

[POSTALCODE] [varchar](20) NULL,

[ContactId] [int] NULL,

[NationalAccount] [bit] NULL,

[TMAssigned] [varchar](30) NULL,

[MaterialBank] [bit] NULL,

[NationalAccountName] [varchar](2000) NULL,
CONSTRAINT [PK_tblOPPERTUNITIESTest] PRIMARY KEY CLUSTERED

(

[OppertunityID] ASC
)WITH (STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]

) ON [PRIMARY]

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

In [8]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [9]:
question = "Find out opportunity name which has invoicePrice between 5000 and 10000"
generated_sql = generate_query(question)
print(generated_sql)


SELECT o.OppertunityName
FROM [PowerAppDev].[tblOPPERTUNITIESTest] o
WHERE o.InvoicePrice BETWEEN 5000 AND 10000
